In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Cargar el DataFrame de pandas
data = pd.read_csv('/content/MAHD.csv')  # Reemplaza 'ruta_del_archivo.csv' con la ubicación real del archivo

# Eliminar columnas no necesarias
data = data.drop('Unnamed: 0', axis=1)


In [2]:
# Definir la clase del conjunto de datos personalizado
class MNISTDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.labels = data['label'].values
        self.features = data.drop('label', axis=1).values.astype('float32')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.labels[idx]
        features = self.features[idx]
        return torch.tensor(features), torch.tensor(label)

In [3]:
# Crear instancias del conjunto de datos de entrenamiento y prueba
dataset = MNISTDataset(data)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

In [4]:
# Definir el modelo del perceptrón
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(784, 10)  # Capa lineal de entrada (784) a salida (10)

    def forward(self, x):
        x = self.fc(x)
        return x

In [5]:
# Crear la instancia del modelo y definir la función de pérdida y el optimizador
model = Perceptron()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Crear los iteradores de carga de datos
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [6]:
# Función de entrenamiento
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

In [7]:
# Función de evaluación
def evaluate(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy}')

In [8]:
# Entrenar y evaluar el modelo
num_epochs = 10
train(model, train_loader, criterion, optimizer, num_epochs)
evaluate(model, test_loader, criterion)

#Cabe recalcar que el dataset que se eligio tiene problemas con la precision de entrenamiento dado que es muy bajo, ya sea utilizando con pytorch o no.

Epoch [1/10], Loss: 4.732202647483512
Epoch [2/10], Loss: 2.7332982684923635
Epoch [3/10], Loss: 2.666160572065066
Epoch [4/10], Loss: 2.631966083017114
Epoch [5/10], Loss: 2.6311165431863097
Epoch [6/10], Loss: 2.6214406653626323
Epoch [7/10], Loss: 2.6131207409514685
Epoch [8/10], Loss: 2.607830497227847
Epoch [9/10], Loss: 2.6087314864816187
Epoch [10/10], Loss: 2.6021258912674368
Test Accuracy: 0.3147857142857143
